#  Sleep and Dream Database (SDDb) Web-Scraping 

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options


import time

## 1. Scroll Through SDDb to Scrap Selected Dream Entries' Links

In [ ]:
path = '/Users/mekdiyilma/code/chromedriver'
driver = webdriver.Chrome(path)

# the Sleep and Dream Database filtered for natural elements: Fire, Water, Earth, Air
url = 'https://sleepanddreamdatabase.org/search_results/%7B%22keywordIds%22%3A[%22%2Fsub_categories%2Fb2de59fe-8b60-11ea-ba75-0242ac120007%22%2C%22%2Fsub_categories%2Fb2de5a34-8b60-11ea-ba75-0242ac120007%22%2C%22%2Fsub_categories%2Fb2de5a69-8b60-11ea-ba75-0242ac120007%22%2C%22%2Fsub_categories%2Fb2de5aa2-8b60-11ea-ba75-0242ac120007%22]%2C%22keywordTexts%22%3A[%22fire%22%2C%22air%22%2C%22water%22%2C%22earth%22]%7D'
driver.get(url)

search = driver.find_element_by_class_name('sc-pJVnX.jOExFg')
time.sleep(3)
previous_height = driver.execute_script('return document.body.scrollHeight')

while True:
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(2)
    new_height = driver.execute_script('return document.body.scrollHeight')
    
    if new_height == previous_height:
        break
    previous_height = new_height


try:
    
    links = WebDriverWait(driver, 120).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'sc-pciXn.kKVZfa'))
    )
    result_lst = []
    for link in links:
        result_lst.append(link.get_attribute('href'))
        
finally:
    driver.quit()

## 2. Save the Links to a Text File

In [ ]:
with open('SDDb_links.txt', 'w') as f:
    for link in result_lst:
        f.write(link + '\n')
        
with open('SDDb_links.txt', 'r') as f:
    links_lst = [line.rstrip('\n') for line in f]

## 3. Fetch Data From Each Dream Entry and Store Result

In [4]:
dreams_data = []
failed_links = []

for link in link_lst:
    chrome_options = Options()
    chrome_options.add_argument("--headless")

    path = '/Users/mekdiyilma/code/chromedriver'
    driver = webdriver.Chrome(executable_path=path, options=chrome_options)
    driver.get(str(link))

    
    try:
        result_dict = dict()
        
        id_search = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'sc-fznAgC.dSEOxJ'))
    )
        for series_id in id_search:
            result_dict['id'] = series_id.text

            
        date_search = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'sc-fzqMAW.himZKo'))
    )
        
        for header in date_search:
            result_dict['date'] = header.text.split('\n')[1]

            
        descrp_search = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'sc-fzoydu.ctZjfV'))
    )

        for descrp in descrp_search:
            result_dict['description'] = descrp.text
        
        try:
            category_search = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, 'sc-fznLPX.dtmoaU'))
        )
            result = []
            for categories in category_search:
                result.append(tuple(categories.text.split('\n')))

            result_dict.update(dict(result))
        except:
            result_dict.update({})
            
        
        dreams_data.append(result_dict)
        
        time.sleep(3)
        
    except:
        failed_links.append(link)
        print('[!] Error: ' + str(link))
